# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
%pip install pymongo

import pandas as pd
from pymongo import MongoClient
client = MongoClient("companies")

Note: you may need to restart the kernel to use updated packages.


In [2]:
cursor=MongoClient()

cursor

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [3]:
cursor.list_database_names()

['admin', 'companies', 'config', 'local', 'sdf']

In [4]:
db=cursor.companies

In [5]:
colec = db.nose

In [9]:
list(colec.find({'name':'Babelgum'},{'name':True}))

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [13]:
list(colec.find({"number_of_employees": {"$gt": 5000}},{'name':True}).limit(20).sort("number_of_employees", 1))

[{'_id': ObjectId('52cdef7c4bab8bd675297e49'), 'name': 'Nintendo'},
 {'_id': ObjectId('52cdef7e4bab8bd67529ba2e'),
  'name': 'Hexaware Technologies'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'), 'name': 'Facebook'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a453'), 'name': 'OpenText'},
 {'_id': ObjectId('52cdef7d4bab8bd675298b89'), 'name': 'LSI'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b9dd'), 'name': 'CPM Braxis'},
 {'_id': ObjectId('52cdef7e4bab8bd67529b691'),
  'name': 'Microchip Technologies'},
 {'_id': ObjectId('52cdef7e4bab8bd67529a84c'), 'name': 'Mediaset'},
 {'_id': ObjectId('52cdef7d4bab8bd675299f38'),
  'name': 'Mindray Medical International'},
 {'_id': ObjectId('52cdef7c4bab8bd675298760'), 'name': 'Baidu'},
 {'_id': ObjectId('52cdef7d4bab8bd675298b6f'), 'name': 'Dentsu'},
 {'_id': ObjectId('52cdef7e4bab8bd67529aeec'), 'name': 'Tata Communications'},
 {'_id': ObjectId('52cdef7e4bab8bd67529bd33'), 'name': 'Atmel'},
 {'_id': ObjectId('52cdef7d4bab8bd675299a81'), 'name': 'Acxiom

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [31]:
list(colec.find({"founded_year": {"$gte": 2000, "$lte": 2005}}, {"name": 1, "founded_year": 1, "_id": 0}).limit(20))

# lo limito a 20 porque salen un montón de resultados

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Skype', 'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [106]:
list(colec.find({"$and": [{"ipo.valuation_amount": {"$gt": 100000000}}, {"founded_year": {"$lt": 2010}}]}, {"name": 1, "ipo": 1, "_id": 0}).limit(2))

# pongo limit dos porque se vuelve enorme

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [108]:
list(colec.find({"$and": [{"number_of_employees": {"$lt": 1000}}, {"founded_year": {"$lt": 2005}}]}, {"name": 1, "number_of_employees": 1, "_id": 0}).sort("number_of_employees", 1).limit(10))


[{'name': 'Fox Interactive Media', 'number_of_employees': 0},
 {'name': 'Ticketmaster', 'number_of_employees': 0},
 {'name': 'Eurekster', 'number_of_employees': 0},
 {'name': 'MindTouch', 'number_of_employees': 0},
 {'name': 'Skype', 'number_of_employees': 0},
 {'name': 'Monster', 'number_of_employees': 0},
 {'name': 'EditGrid', 'number_of_employees': 0},
 {'name': 'Simpy', 'number_of_employees': 0},
 {'name': 'Compete', 'number_of_employees': 0},
 {'name': 'stylediary', 'number_of_employees': 0}]

### 6. All the companies that don't include the `partners` field.

In [109]:
list(colec.find({'partners': {'$nin': [0]}}, {'name': True, '_id': False}).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [61]:
list(colec.find({"category_code": None},{'name':True}).limit(7))

[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'), 'name': 'Collective'},
 {'_id': ObjectId('52cdef7c4bab8bd675298225'), 'name': 'Snimmer'},
 {'_id': ObjectId('52cdef7c4bab8bd675298226'), 'name': 'KoolIM'},
 {'_id': ObjectId('52cdef7c4bab8bd675298261'), 'name': 'Level9 Media'},
 {'_id': ObjectId('52cdef7c4bab8bd675298262'), 'name': 'VidKing'},
 {'_id': ObjectId('52cdef7c4bab8bd67529826e'), 'name': 'Drigg'},
 {'_id': ObjectId('52cdef7c4bab8bd675298276'), 'name': 'SpaceTime'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [110]:
list(colec.find({"$and": [{"number_of_employees": {"$gte": 100}}, {"number_of_employees": {"$lt": 1000}}]}, {"name": 1, "number_of_employees": 1, "_id":0}).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [63]:
# dice Yona que no nos rayemos con este, que pasemos al siguiente

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [71]:
list(colec.find({},{"name": 1, "number_of_employees": 1, "_id":0}).sort("number_of_employees", -1).limit(10))

# le digo que me muestre sólo las columnas name y number of employees

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [72]:
list(colec.find({
    "founded_month": {"$gte": 7},
    "founded_month": {"$lte": 12}
}, {"name": 1, "_id":0}).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Twitter'},
 {'name': 'StumbleUpon'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [77]:
list(colec.find({"founded_year": {"$lt": 2000},"acquisition.price_amount": {"$gt": 10000000}}, {"name": 1, "_id": 0}).limit(5))

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [90]:
docs = colec.find({"acquisition.acquired_year": {"$gt": 2010}}, {"name": 1, "acquisition.price_amount": 1}).sort("acquisition.price_amount", -1).limit(10)
list(docs)

[{'_id': ObjectId('52cdef7c4bab8bd675298876'),
  'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0}},
 {'_id': ObjectId('52cdef7f4bab8bd67529c228'),
  'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0}},
 {'_id': ObjectId('52cdef7d4bab8bd675298b89'),
  'name': 'LSI',
  'acquisition': {'price_amount': 6600000000.0}},
 {'_id': ObjectId('52cdef7e4bab8bd67529a2b5'),
  'name': 'National Semiconductor',
  'acquisition': {'price_amount': 6500000000.0}},
 {'_id': ObjectId('52cdef7d4bab8bd675298935'),
  'name': 'Ariba',
  'acquisition': {'price_amount': 4300000000.0}},
 {'_id': ObjectId('52cdef7f4bab8bd67529bff7'),
  'name': 'NetLogic Microsystems',
  'acquisition': {'price_amount': 3700000000.0}},
 {'_id': ObjectId('52cdef7c4bab8bd6752981d6'),
  'name': 'SuccessFactors',
  'acquisition': {'price_amount': 3400000000.0}},
 {'_id': ObjectId('52cdef7e4bab8bd67529aba6'),
  'name': 'Qualcomm Atheros',
  'acquisition': {'price_amount': 3100000000.0}},
 

In [112]:
list(colec.find({"acquisition.acquired_year": {"$gt": 2010}}, {"name": 1, "acquisition": 1, "_id": 0}).sort("acquisition.price_amount", 1).limit(3))


[{'name': 'blogTV',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2013/03/13/younow-buys-blogtv/',
   'source_description': 'Live Social Video Network YouNow Acquires Streaming Service BlogTV',
   'acquired_year': 2013,
   'acquired_month': 3,
   'acquired_day': 13,
   'acquiring_company': {'name': 'YouNow', 'permalink': 'younow'}}},
 {'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [114]:
docs = colec.find({}, {"name": 1, "founded_year": 1}).sort("founded_year", 1).limit(10)

for doc in docs:
    print(doc["name"], doc["founded_year"])

    

# hago esto pero me salen las fechas de fundación como None. Puede ser porque ese elemento esté vacío?

Info None
Lala None
NetRatings None
JotSpot None
Flektor None
Tun3r None
There None
SpinVox None
Mercora None
Gannett None


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [96]:
list(colec.find({"founded_day": {"$lte": 7}}, {"_id": 0, "name": 1, "acquisition.price_amount": 1}).sort("acquisition.price_amount", -1).limit(10))


[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000.0}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger', 'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems', 'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense', 'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee', 'acquisition': {'price_amount': 321000000}},
 {'name': 'Vitrue', 'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [99]:
list(colec.find({"category_code": "web", "number_of_employees": {"$gt": 4000}}, {"_id": 0, "name": 1, "number_of_employees": 1}).sort("number_of_employees", 1).limit(10))


[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [100]:
list(colec.find({"acquisition.price_amount": {"$gt": 10000000}, "acquisition.price_currency_code": "EUR"}, {"_id": 0, "name": 1, "acquisition": 1}).limit(3))


[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000,
   'price_currency_code': 'EUR',
   'term_code': 'cash',
   'source_url': 'http://www.techcrunch.com/2008/05/16/danish-mobile-social-network-zyb-acquired-by-vodafone-for-e315-million/',
   'source_description': 'Danish Network ZYB Acquired By Vodafone For â‚¬31.5 million',
   'acquired_year': 2008,
   'acquired_month': 5,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Vodafone', 'permalink': 'vodafone'}}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000,
   'price_currency_code': 'EUR',
   'term_code': None,
   'source_url': 'http://www.techcrunch.com/2008/01/03/nokia-siemens-acquires-apertio-for-e140m/',
   'source_description': 'Nokia Siemens Acquires Apertio For â‚¬140m',
   'acquired_year': 2008,
   'acquired_month': 1,
   'acquired_day': 3,
   'acquiring_company': {'name': 'Nokia Siemens Networks',
    'permalink': 'nokia-siemens-networks'}}},
 {'name': 'Greenfield Online',
  'acquisition': {'p

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [115]:
list(colec.find({"acquisition.acquired_month": {"$lte": 3}}, {"_id": 0, "name": 1, "acquisition": 1}).sort("acquisition.price_amount", -1).limit(5))


[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'BEA Systems',
  'acquisition': {'price_amount': 8500000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.oracle.com/corporate/press/2008_jan/bea.html',
   'source_description': 'Oracle to Acquire BEA Systems',
   'acquired_year': 2008,
   'acquired_month': 1,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Oracle Corporation',
    'permalink': 'oracle'}}},
 {'name': 'Puget Energy',
  'acquisition': {'price_amount': 7400000000.0,
   'price_curre

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code